In [1]:
import os
import json
import time
import openai
from underthesea import sent_tokenize
from langchain_text_splitters import CharacterTextSplitter
from tqdm import tqdm

# Thiết lập API key cho OpenAI từ biến môi trường
openai.api_key = os.getenv('OPENAI_API_KEY')

# Kiểm tra xem API key có được thiết lập hay không
if openai.api_key is None:
    raise ValueError("API key chưa được thiết lập. Vui lòng thiết lập biến môi trường 'OPENAI_API_KEY'.")

class VietnameseTextProcessor:
    def __init__(self, max_length: int = 5000):
        self.max_length = max_length

    def read_text_files(self, input_folder: str) -> str:
        """Đọc tất cả các file văn bản trong thư mục."""
        all_texts = []
        for filename in os.listdir(input_folder):
            if filename.endswith('.txt'):
                with open(os.path.join(input_folder, filename), 'r', encoding='utf-8') as f:
                    all_texts.append(f.read())
        return "\n".join(all_texts)

    def create_chunks(self, text: str) -> list:
        """Tạo chunk từ văn bản."""
        splitter = CharacterTextSplitter(
            chunk_size=self.max_length,
            chunk_overlap=500,
            separator="\n",
            length_function=len
        )
        return splitter.split_text(text)

    def generate_questions(self, chunk: str) -> list:
        """Gửi chunk tới GPT-4o-mini để tạo câu hỏi."""
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Bạn am hiểu về Phật Pháp, Thiền Học của Thiền Sư Thích Nhất Hạnh."},
                {"role": "user", "content": f"""
                    Từ "nội dung sách", hãy đặt 5 câu hỏi.
                    Đóng vai trò là Phật Tử tu học, muốn được Thiền Sư Thích Nhất Hạnh giảng dạy để đặt câu hỏi.
                    Các câu hỏi đảm bảo rõ ràng, cụ thể, phản ánh và khai thác đầy đủ nội dung sách.
                    Hãy trả về kết quả dưới dạng JSON như sau:
                    {{
                        "questions": [
                            "Câu hỏi 1",
                            "Câu hỏi 2"
                        ]
                    }}
                    Nội dung sách: {chunk}
                """},
            ],
            temperature=0.5,
            response_format={"type": "json_object"}
        )

        questions_json = response.choices[0].message.content.strip()
        
        # print("Nội dung phản hồi từ API:", questions_json)  # Thêm dòng này để kiểm tra

        # Kiểm tra xem nội dung có hợp lệ không
        if not questions_json:
            raise ValueError("Nội dung phản hồi trống.")

        # Chuyển đổi chuỗi JSON thành danh sách
        try:
            questions = json.loads(questions_json)["questions"]
        except json.JSONDecodeError as e:
            print("Lỗi phân tích cú pháp JSON:", e)
            raise ValueError("Nội dung phản hồi không phải là JSON hợp lệ.")

        return questions

    def create_samples(self, questions: list, chunk: str) -> list:
        """Tạo sample theo định dạng Alpaca."""
        samples = []
        for question in questions:
            sample = {
                "instruction": "Hãy trả lời theo văn phong của Thiền Sư Thích Nhất Hạnh.",
                "input": question,
                "output": chunk
            }
            samples.append(sample)
        return samples

def main():
    processor = VietnameseTextProcessor(max_length=5000)

    input_folder = "data-raw/Thich Nhat Hanh 2"
    output_file = "output_samples.json"

    all_samples = []
    total_tokens = 0
    total_samples_processed = 0

    # Đọc tất cả văn bản từ các file trong thư mục
    full_text = processor.read_text_files(input_folder)

    # Tạo chunks từ văn bản
    chunks = processor.create_chunks(full_text)

    start_time = time.time()

    # Đơn giá cho input và output tokens
    input_price_per_million = 0.150 
    output_price_per_million = 0.600 

    # Khởi tạo tổng số token đầu vào và đầu ra
    total_request_tokens = 0
    total_response_tokens = 0

    # Xử lý từng chunk và tạo câu hỏi
    for chunk in tqdm(chunks, desc="Processing chunks"):
        questions = processor.generate_questions(chunk)

        # Tính toán số token cho request và response
        request_tokens = len(chunk.split()) + sum(len(q.split()) for q in questions)
        response_tokens = sum(len(q.split()) for q in questions)

        total_request_tokens += request_tokens
        total_response_tokens += response_tokens

        total_tokens += (request_tokens + response_tokens)

        # Tạo sample cho từng câu hỏi
        samples = processor.create_samples(questions, chunk)
        all_samples.extend(samples)

        total_samples_processed += len(samples)

    # Tính toán chi phí dựa trên số token đã sử dụng
    input_cost = (total_request_tokens / 1000000) * input_price_per_million
    output_cost = (total_response_tokens / 1000000) * output_price_per_million
    total_cost = input_cost + output_cost

    # Lưu tất cả sample vào file JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_samples, f, ensure_ascii=False, indent=2)

    end_time = time.time()

    print(f"\nXử lý xong:")
    print(f"- Tổng số mẫu đã xử lý: {total_samples_processed}")
    print(f"- Tổng số token đã sử dụng: {total_tokens}")
    print(f"- Tổng số input tokens: {total_request_tokens}")
    print(f"- Tổng số output tokens: {total_response_tokens}")
    print(f"- Chi phí cho input tokens: ${input_cost:.3f}")
    print(f"- Chi phí cho output tokens: ${output_cost:.3f}")
    print(f"- Tổng chi phí: ${total_cost:.3f}")
    print(f"- Thời gian thực hiện: {end_time - start_time:.2f} giây")
    print(f"- Dataset được lưu tại: {output_file}")

if __name__ == "__main__":
    main()

/Users/phucnt/.pyenv/versions/3.11.10/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing chunks: 100%|██████████| 580/580 [27:21<00:00,  2.83s/it]


Xử lý xong:
- Tổng số mẫu đã xử lý: 2900
- Tổng số token đã sử dụng: 772246
- Tổng số input tokens: 687577
- Tổng số output tokens: 84669
- Chi phí cho input tokens: $0.103
- Chi phí cho output tokens: $0.051
- Tổng chi phí: $0.154
- Thời gian thực hiện: 1642.03 giây
- Dataset được lưu tại: output_samples.json
